In [108]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [109]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [110]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [111]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [112]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file

X = df_market_data.iloc[:, :-1].values
y = df_market_data.iloc[:, -1].values

scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
df_scaled = pd.DataFrame(X_scaled, columns=df_market_data.columns[:-1])




In [118]:
# create a new DataFrame with the scaled data
df_scaled = pd.DataFrame(X_scaled, columns=df_market_data.columns[1:-1])

# copy the coinid column from the original data
df_scaled['coin_id'] = df_market_data['coin_id']

# set the coinid column as the index
df_scaled.set_index('coin_id', inplace=True)

# display the first few rows of the scaled data
print(df_scaled.head())


              price_change_percentage_24h  price_change_percentage_7d  \
coin_id                                                                 
bitcoin                          0.508529                    0.493193   
ethereum                         0.185446                    0.934445   
tether                           0.021774                   -0.706337   
ripple                          -0.040764                   -0.810928   
bitcoin-cash                     1.193036                    2.000959   

              price_change_percentage_14d  price_change_percentage_30d  \
coin_id                                                                  
bitcoin                          0.772200                     0.235460   
ethereum                         0.558692                    -0.054341   
tether                          -0.021680                    -0.061030   
ripple                           0.249458                    -0.050388   
bitcoin-cash                     1.760610   

---

### Find the Best Value for k Using the Original Data.

In [119]:
# Create a list with the number of k-values from 1 to 11

k_values = list(range(1, 12))

# display the list of k-values
print(k_values)


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [120]:
# Create an empty list to store the inertia values
inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
for k in k_values:
    kmeans_model = KMeans(n_clusters=k)
    kmeans_model.fit(df_scaled)
    # Append the inertia to the list
    inertia_values.append(kmeans_model.inertia_)

# 1. Create a KMeans model using the loop counter for the n_clusters

kmeans_model = KMeans(n_clusters=k)

# 2. Fit the model to the data using `df_market_data_scaled`
kmeans_model.fit(df_scaled)

# 3. Append the model.inertia_ to the inertia list


/Users/Crisaldry/opt/anaconda3/envs/PythonData/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/Crisaldry/opt/anaconda3/envs/PythonData/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/Crisaldry/opt/anaconda3/envs/PythonData/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/Crisaldry/opt/anaconda3/envs/PythonData/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Se

KMeans(n_clusters=11)

In [121]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {'k': k_values, 'inertia': inertia_values}


# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

df_elbow

,k,inertia
0,1,246.000000
1,2,172.987224
2,3,124.018500
3,4,78.336283
4,5,65.746831
5,6,51.989409
6,7,42.751558
7,8,36.301172
8,9,32.356726
9,10,27.766296


In [122]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.

df_elbow.hvplot.line(x='k', y='inertia', title='Elbow Curve', xticks=k_values)


:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:**
The best value for K is 11 as it is the one with the smallest inertia value.The smaller the inertia value the more compact the clusters.  

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [123]:
# Initialize the K-Means model using the best value for k

k = 11 
kmeans_model = KMeans(n_clusters=k)
kmeans_model.fit(df_scaled)


/Users/Crisaldry/opt/anaconda3/envs/PythonData/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=11)

In [124]:
# Fit the K-Means model using the scaled data
kmeans_model.fit(df_scaled)


/Users/Crisaldry/opt/anaconda3/envs/PythonData/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=11)

In [125]:
# Predict the clusters to group the cryptocurrencies using the scaled data

cluster_labels = kmeans_model.predict(df_scaled)

# Print the resulting array of cluster values.
cluster_labels = kmeans_model.labels_
print(cluster_labels)


[10 10  1  1  4  4  0  0 10  1  5  1  1  4  1 10  5  1  0  1  5 10  5  1
  9  1  1  8 10  7  1  1  3 10  8  6  2  8 10  1  5]


In [126]:
# Create a copy of the DataFrame
df_cluster = df_scaled.copy().assign(cluster=kmeans_model.labels_)



In [129]:
# Add a new column to the DataFrame with the predicted clusters


# Display sample data
df_cluster

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,cluster
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,10
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,10
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,1
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,1
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,4
binancecoin,0.891871,1.327295,0.800214,-0.057148,0.778653,-0.188232,4
chainlink,0.011397,2.572251,1.101647,-0.490495,-0.931954,0.387759,0
cardano,0.102530,1.508001,0.648885,0.328959,-0.486349,0.065080,0
litecoin,0.077497,0.334297,0.858520,-0.012646,-0.366477,-0.486266,10


In [130]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.


# Set the x and y axis values
x_axis = "price_change_percentage_24h"
y_axis = "price_change_percentage_7d"

# Create the scatter plot
scatter_plot = df_cluster.hvplot.scatter(
    x=x_axis,
    y=y_axis,
    by="cluster",
    hover_cols=["currency"],
    title="Cryptocurrency Clusters",
)

# Show the plot
scatter_plot



:NdOverlay   [cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d)

---

### Optimize Clusters with Principal Component Analysis.

In [131]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [132]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
pca_features = pca.fit_transform(df_scaled)

# View the first five rows of the DataFrame. 
df_scaled.head(5)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d
coin_id,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848


In [133]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
explained_variance = pca.explained_variance_ratio_
print(explained_variance)

pca.explained_variance_ratio_.sum()


[0.40706509 0.29908902 0.18491371]


0.8910678142433394

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 

In [134]:
df_scaled["coin_id"] = df_market_data["coin_id"]

In [135]:
print(df_scaled.columns)

Index(['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d',
       'price_change_percentage_60d', 'price_change_percentage_200d',
       'coin_id'],
      dtype='object')


In [136]:
# copy the crypto names from the original data
crypto_names = df_market_data["coin_id"].copy()

# create a new DataFrame with the PCA data and crypto names
pca_df["coin_id"] = crypto_names
pca_df = pca_df.set_index("coin_id")

# display the first 5 rows of the new DataFrame
pca_df.head()


,PC 1,PC 2,PC 3
coin_id,,,
bitcoin,0.614827,-0.913141,0.182100
ethereum,0.289898,-0.789507,0.766643
tether,-0.282072,-0.065321,-0.678397
ripple,-0.346857,-0.124809,-0.520935
bitcoin-cash,1.565813,-2.297316,1.149002


---

### Find the Best Value for k Using the PCA Data

In [138]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))

k_values

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [139]:
# Create an empy list to store the inertia values
inertia_list = []


# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list

for k in range(1, 12):
   kmeans_model = KMeans(n_clusters=k, random_state=0)
    # Fit the model to the data using `df_market_data_pca`
   kmeans_model.fit(pca_df)
    # Append the model.inertia_ to the inertia list
   inertia_list.append(kmeans_model.inertia_) 


/Users/Crisaldry/opt/anaconda3/envs/PythonData/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/Crisaldry/opt/anaconda3/envs/PythonData/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/Crisaldry/opt/anaconda3/envs/PythonData/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/Crisaldry/opt/anaconda3/envs/PythonData/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Se

In [140]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k_values, "inertia": inertia_list}
# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_data)

elbow_df

,k,inertia
0,1,219.202682
1,2,146.533566
2,3,96.202520
3,4,52.083243
4,5,40.011905
5,6,29.055853
6,7,22.430600
7,8,18.536277
8,9,14.783348
9,10,11.154218


In [141]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xlabel="k", ylabel="inertia")

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 10 is the best value for K, because while K is 10, the inertia is at its owest point.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** With the original data, the best value for K was 11. Even so, the same logic applies, where the higher the K value, the lower the inertia. 

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [143]:
# Initialize the K-Means model using the best value for k
optimal_k = 10

kmeans_model = KMeans(n_clusters=optimal_k)


In [144]:
# Fit the K-Means model using the PCA data

kmeans_model.fit(pca_df)


/Users/Crisaldry/opt/anaconda3/envs/PythonData/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=10)

In [146]:
# Predict the clusters to group the cryptocurrencies using the PCA data
# Print the resulting array of cluster values.

kmeans = KMeans(n_clusters=optimal_k, random_state=0)
crypto_clusters = kmeans.fit_predict(pca_df)
print(crypto_clusters)






[0 0 4 4 6 0 7 7 0 9 2 4 4 6 9 9 2 9 7 4 2 0 4 4 4 4 9 2 0 1 9 9 5 9 8 4 3
 2 9 4 4]


/Users/Crisaldry/opt/anaconda3/envs/PythonData/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [152]:
# Create a copy of the DataFrame with the PCA data

clustered_df = pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
clustered_df["cluster"] = kmeans.labels_

# Display sample data
clustered_df.head()




,PC 1,PC 2,PC 3,cluster
coin_id,,,,
bitcoin,0.614827,-0.913141,0.182100,0
ethereum,0.289898,-0.789507,0.766643,0
tether,-0.282072,-0.065321,-0.678397,4
ripple,-0.346857,-0.124809,-0.520935,4
bitcoin-cash,1.565813,-2.297316,1.149002,6


In [156]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.


final_df = pd.merge(clustered_df, df_market_data[["coin_id", "price_change_percentage_24h", "price_change_percentage_7d"]], on="coin_id")

# Create scatter plot
scatter_plot = final_df.hvplot.scatter(x="price_change_percentage_24h", y="price_change_percentage_7d", by="cluster", hover_cols=["coin_id"])
scatter_plot




:NdOverlay   [cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [158]:
# Composite plot to contrast the Elbow curves

# elbow curve using the original data
elbow_curve = pd.DataFrame({
    "k": k_values,
    "inertia": inertia_list
}).hvplot.line(x="k", y="inertia", title="Elbow Curve - Original Data")

# elbow curve using the PCA data
elbow_curve_pca = pd.DataFrame({
    "k": k_values,
    "inertia": inertia_list
}).hvplot.line(x="k", y="inertia", title="Elbow Curve - PCA Data")

# composite plot
composite_plot = elbow_curve + elbow_curve_pca

composite_plot


:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [165]:
# Composite plot to contrast the clusters
# Create scatter plot for original data
scatter_original = final_df.hvplot.scatter(
    x="price_change_percentage_24h", 
    y="price_change_percentage_7d", 
    by="cluster", 
    hover_cols=["coin_id"],
    title="Original Data"
)

# Create scatter plot for PCA data
scatter_pca = clustered_df.hvplot.scatter(
    x="price_change_percentage_24h", 
    y="price_change_percentage_7d", 
    by="cluster", 
    hover_cols=["coin_id"],
    title="PCA Data"
)

# Create composite plot
composite_plot = scatter_original + scatter_pca

# Show the plot
composite_plot


DataError: Supplied data does not contain specified dimensions, the following dimensions were not found: ['price_change_percentage_24h', 'price_change_percentage_7d']

PandasInterface expects tabular data, for more information on supported datatypes see http://holoviews.org/user_guide/Tabular_Datasets.html

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** 